In [6]:
# 导入必备依赖
import argparse
import os
import numpy as np
import math

# 导入图像数据增广对象
import torchvision.transforms as transforms

# 导入图像保存对象
from torchvision.utils import save_image

# 导入数据集加载对象
from torch.utils.data import DataLoader

# 导入torch自带数据集，稍后从中下载并使用MNIST数据集
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [7]:
# 创建保存生成图像的文件夹
os.makedirs("./images", exist_ok=True)

epoches = 50 # 总训练轮数；考虑到没必要面向生产，仅作为算法学习使用的简单demo，出于节约成本的考量，训练轮数仅设计为50轮即可
batch_size = 64 # 一个训练step输入的sample数量
init_lr = 0.0002 # Adam优化器初始化学习率
b1 = 0.5 # Adam优化器的第一个动量衰减参数
b2 = 0.999 # Adam优化器的第二个动量衰减参数
n_cpu = 8 # 用于一次生成的CPU线程数
latent_dim = 100 # 候选空间的维度
num_classes = 10 # sample的label的类别总数
img_size = 32 # 单个输入图像的尺寸 - 宽高相等
channels = 1 # 输入图像的通道数 - 灰度图，单通道即可
sample_interval = 400 # 图像采样间隔

# 如果 GPU 可用，则使用 CUDA 加速
cuda = True if torch.cuda.is_available() else False
cuda

True

In [8]:
# 初始化模型权重
def weights_init_normal(m):
    '''
    传入model，初始化model的权重
    '''
    
    # 通过传入的model获取其类名
    classname = m.__class__.__name__
    
    # 如果是卷积网络
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    
    # 否则就不是
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [9]:
# 定义生成器对象 - 这里就是DCGAN的Generator
class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()

        # 创建一个标签嵌入层，用于将条件标签映射到潜在空间
        self.label_emb = nn.Embedding(num_classes, latent_dim)

        # 初始化图像尺寸，用于上采样之前
        self.init_size = img_size // 4  # Initial size before upsampling

        # 第一个全连接层，将随机噪声映射到合适的维度
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        # 生成器的卷积块
        self.conv_blocks = nn.Sequential(
            
            # BN纵向规范
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.l1(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """返回每个鉴别器块的层"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        # 鉴别器的卷积块
        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # 下采样图像的高度和宽度
        ds_size = img_size // 2 ** 4

        # 输出层
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())  # 用于鉴别真假的输出层
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, num_classes + 1), nn.Softmax())  # 用于鉴别类别的输出层

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label

In [11]:
# 定义损失函数
adversarial_loss = torch.nn.BCELoss()  # 二元交叉熵损失，用于对抗训练
auxiliary_loss = torch.nn.CrossEntropyLoss()  # 交叉熵损失，用于辅助分类

# 初始化生成器和鉴别器
generator = Generator()  # 创建生成器实例
discriminator = Discriminator()  # 创建鉴别器实例

# 如果使用GPU，将模型和损失函数移至GPU上
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# 初始化模型权重
generator.apply(weights_init_normal)  # 初始化生成器的权重
discriminator.apply(weights_init_normal)  # 初始化鉴别器的权重

# 配置数据加载器
os.makedirs("./data/mnist", exist_ok=True)  # 创建存储MNIST数据集的文件夹
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

# 优化器
optimizer_G = torch.optim.Adam(generator.parameters(), lr=init_lr, betas=(b1, b2))  # 生成器的优化器
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=init_lr, betas=(b1, b2))  # 鉴别器的优化器

# 根据是否使用GPU选择数据类型
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [12]:
for epoch in range(epoches):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # 定义对抗训练的标签
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)  # 用于真实样本
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)  # 用于生成样本
        fake_aux_gt = Variable(LongTensor(batch_size).fill_(num_classes), requires_grad=False)  # 用于生成样本的类别标签

        # 配置输入数据
        real_imgs = Variable(imgs.type(FloatTensor))  # 真实图像
        labels = Variable(labels.type(LongTensor))  # 真实类别标签

        # -----------------
        #  训练生成器
        # -----------------

        optimizer_G.zero_grad()

        # 采样噪声和类别标签作为生成器的输入
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))

        # 生成一批图像
        gen_imgs = generator(z)

        # 计算生成器的损失，衡量生成器欺骗鉴别器的能力
        validity, _ = discriminator(gen_imgs)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  训练鉴别器
        # ---------------------

        optimizer_D.zero_grad()

        # 真实图像的损失
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # 生成图像的损失
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, fake_aux_gt)) / 2

        # 总的鉴别器损失
        d_loss = (d_real_loss + d_fake_loss) / 2

        # 计算鉴别器准确率
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), fake_aux_gt.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "./images/%d.png" % batches_done, nrow=5, normalize=True)

    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
        % (epoch, epoches, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
    )

/home/ma-user/anaconda3/envs/PyTorch-2.0.0/lib/python3.9/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 0/50] [Batch 937/938] [D loss: 1.368545, acc: 50%] [G loss: 0.704623]
[Epoch 1/50] [Batch 937/938] [D loss: 1.387643, acc: 50%] [G loss: 0.786353]
[Epoch 2/50] [Batch 937/938] [D loss: 1.369781, acc: 50%] [G loss: 0.674227]
[Epoch 3/50] [Batch 937/938] [D loss: 1.345224, acc: 51%] [G loss: 0.772573]
[Epoch 4/50] [Batch 937/938] [D loss: 1.339223, acc: 51%] [G loss: 0.747099]
[Epoch 5/50] [Batch 937/938] [D loss: 1.287006, acc: 51%] [G loss: 0.851946]
[Epoch 6/50] [Batch 937/938] [D loss: 1.357881, acc: 53%] [G loss: 1.010694]
[Epoch 7/50] [Batch 937/938] [D loss: 1.415346, acc: 45%] [G loss: 0.834191]
[Epoch 8/50] [Batch 937/938] [D loss: 1.344990, acc: 53%] [G loss: 0.821848]
[Epoch 9/50] [Batch 937/938] [D loss: 1.357066, acc: 57%] [G loss: 0.760246]
[Epoch 10/50] [Batch 937/938] [D loss: 1.218984, acc: 50%] [G loss: 0.914214]
[Epoch 11/50] [Batch 937/938] [D loss: 1.251535, acc: 50%] [G loss: 1.077527]
[Epoch 12/50] [Batch 937/938] [D loss: 1.388343, acc: 53%] [G loss: 0.7184